# Annotation and genotyping of tandem repeats in *Arabidopsis thaliana* 

By Adam English (Adam.English@bcm.edu) and Egor Dolzhenko (edolzhenko@pacificbiosciences.com)

In [46]:
import os
import sys
import json
import scipy
import pathlib
import shutil
import numpy as np
from pandas import DataFrame
import matplotlib.pyplot as plt
import matplotlib.patches as patches

%matplotlib inline
plt.rcParams['figure.figsize'] = [16.5, 5]
plt.rcParams['font.size'] = 12

## Prepare inputs

In [63]:
# TAIR10 reference genome
genome_path = "input/TAIR10.fa"

# Simple repeats from UCSC
simple_repeats_path = "input/simple_repeats.bed"

# A HiFi BAM file aligned to TAIR10 (https://ngdc.cncb.ac.cn/gsa/browse/CRA004538/CRR302668)
reads_path = "input/CRR302668.bam"

# A path to TRGT binary
trgt="/home/edolzhenko/flash/projects/trgt/trgt/target/x86_64-unknown-linux-gnu/release/trgt"

assert pathlib.Path(genome_path).is_file(), "The genome file is missing"
assert pathlib.Path(simple_repeats_path).is_file(), "The simple repeats file is missing"
assert pathlib.Path(reads_path).is_file(), "BAM file with reads is missing"
assert shutil.which("trf"), "TRF is not installed"
assert shutil.which("samtools"), "Samtools is not installed"
assert shutil.which("bedtools"), "Bedtools is not installed"
assert shutil.which(trgt), "TRGT is not installed"

In [40]:
! mkdir -p scratch output

## Index the reference

In [33]:
! samtools faidx {genome_path}

## Merging intervals
- Intra-merge the input regions
- Filter to those between 10 and 50kbp in span
- For Arabidopsis, clean the chromosome name to upper case
- Also removing ChrCp, renaming ChrMt to chrM
- Slop the boundaries ±25bp
- Remerge the files (boundaries might cause some to overlap

In [38]:
%%bash -s {simple_repeats_path} {genome_path}

simple_repeats=$1
genome=$2

bedtools merge -i $simple_repeats \
    | awk '($3 - $2) >= 10 && ($3 - $2) < 50000' \
    | awk '{print toupper(substr($0, 1, 1)) substr( $0, 2 );}' \
    | bedtools sort -i - \
    | grep -v ChrCp | sed 's/ChrMt/ChrM/' \
    | bedtools slop -b 25 -g $genome.fai \
    | bedtools merge \
    > scratch/candidates.bed

In [39]:
! head scratch/candidates.bed

Chr1	0	131
Chr1	4275	4357
Chr1	6541	6693
Chr1	8643	8724
Chr1	16600	16676
Chr1	17010	17503
Chr1	37710	37802
Chr1	41335	41410
Chr1	55848	56308
Chr1	62321	62397


## Annotating sequence

In [42]:
%%bash -s {genome_path}

genome=$1

# Extract the candidate region sequences
# Adding 1 since the Arabidopsis Simple Repeats are 0-based
samtools faidx -r <(awk '{print $1 ":" $2 + 1 "-" $3}' scratch/candidates.bed) $genome > scratch/tr_regions.fasta

# Run TRF and reformat for TRGT
trf scratch/tr_regions.fasta 3 7 7 80 5 5 500 -h -ngs > scratch/trf_output.txt

## Convert TRF outputs into 0-based bed files

In [47]:
def parse_trf_output(tr_fn):
    """
    Reads the output from tandem repeat finder
    translates the coordinates from the 'samtools faidx' fetched sequence header 
    back to whole genome coordinates

    returns hits as a locus
    """
    trf_cols = [("start", int), ("end", int), ("period", int), ("copies", float), 
                ("consize", int), ("pctmat", int), ("pctindel", int), ("score", int), 
                ("A", int), ("C", int), ("G", int), ("T",  int), ("entropy", float),
                ("repeat", str), ("upflank", str), ("sequence", str), ("dnflank", str)]
    with open(tr_fn, 'r') as fh:
        name = fh.readline()
        if name == "":  # no hits
            return
        name = name.strip()[1:]
        chrom, coords = name.split(':')
        wgs_start, wgs_end = coords.split('-')
        # 0-based correction
        wgs_start = int(wgs_start) - 2 
        # -1 because faidx is 1-based (found in annotation header)
        # & -1 because trf start is 1-based (found in start)
        wgs_end = int(wgs_end)
        cur_hits = []
        while True:
            line = fh.readline()
            if line == "":
                break
            if line.startswith("@"):
                yield cur_hits
                cur_hits = []
                name = line.strip()[1:]
                chrom, coords = name.split(':')
                wgs_start, wgs_end = coords.split('-')
                wgs_start = int(wgs_start) - 2 # 0-based correction
                wgs_end = int(wgs_end)
                continue
            line = line.strip().split(' ')
            data = {x[0]: x[1](y) for x, y in zip(
                trf_cols, line) if x[1] is not None}
            data['chrom'] = chrom
            data['in_region_start'] = wgs_start
            data['in_region_end'] = wgs_end
            data['start'] += wgs_start
            data['end'] += wgs_start
            cur_hits.append(data)
        yield cur_hits

In [93]:
chrom_lens = ! cat input/TAIR10.fa.fai
chrom_lens = [rec.split()[:2] for rec in chrom_lens]
chrom_lens = {chrom: int(length) for chrom, length in chrom_lens}
chrom_lens

{'Chr1': 30427671,
 'Chr2': 19698289,
 'Chr3': 23459830,
 'Chr4': 18585056,
 'Chr5': 26975502,
 'ChrC': 154478,
 'ChrM': 367808}

In [99]:
def untangle_locus(data):
    """
    Given a set of hits, get min start, max end and motif of longest spanning hit
    """
    chrom = None
    min_start = sys.maxsize
    max_end = 0
    longest_span = 0
    seq = None

    for i in data:
        chrom = i['chrom']
        if i['start'] < min_start:
            min_start = i['start']
        if i['end'] > max_end:
            max_end = i['end']
        span = i['end'] - i['start']
        if span > longest_span:
            longest_span = span
            seq = i['repeat']
    if min_start <= 1000 or abs(max_end - chrom_lens[chrom]) < 1000:
        return None
    fourth_col = f"ID={chrom}_{min_start}_{max_end};MOTIFS={seq};STRUC=({seq})n"
    return f"{chrom}\t{min_start}\t{max_end}\t{fourth_col}\n"

In [100]:
with open("output/tair10_catalog.bed", "w") as fout:
    for locus in parse_trf_output("scratch/trf_output.txt"):
        rec = untangle_locus(locus)
        if rec != None:
            fout.write(rec)

In [ ]:
%%bash -s $trgt $genome_path $reads_path

trgt=$1
genome=$2
reads=$3

sample=$(basename $reads)
sample=${sample%.bam}

$trgt \
  --genome $genome \
  --reads $reads \
  --repeats output/tair10_catalog.bed \
  --output-prefix output/$sample